In [1]:
import os
import mlflow
from dotenv import load_dotenv
load_dotenv()

# определяем основные credentials, которые нужны для подключения к MLflow
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "https://storage.yandexcloud.net" #endpoint бакета от YandexCloud
os.environ["AWS_ACCESS_KEY_ID"] = os.getenv("AWS_ACCESS_KEY_ID") # получаем id ключа бакета, к которому подключён MLFlow, из .env
os.environ["AWS_SECRET_ACCESS_KEY"] = os.getenv("AWS_SECRET_ACCESS_KEY") # получаем ключ бакета, к которому подключён MLFlow, из .env

# определяем глобальные переменные # поднимаем MLflow локально
TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000

# название эксперимента и запуска (run) внутри него
EXPERIMENT_NAME = "bvv_43"
RUN_NAME = "log_model"

# устанавливаем host, который будет отслеживать наши эксперименты
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_registry_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}") 

In [3]:
import pandas as pd
import joblib
#from sklearn.compose import ColumnTransformer
#from sklearn.pipeline import Pipeline
#from sklearn.preprocessing import StandardScaler, OneHotEncoder
#from catboost import CatBoostClassifier
#pip show category_encoders #2.6.3
#pip show scikit-learn #1.4

# прочитаем данные и обученную модель из 1 спринта
data = pd.read_csv('data/initial_data.csv')    
with open('models/fitted_model.pkl', 'rb') as fd:
    model = joblib.load(fd)

In [4]:
from sklearn.metrics import auc, precision_score, recall_score, f1_score,log_loss, confusion_matrix

val = data.head(20)
y_test= val.target
prediction = model.predict(val)
probas = model.predict_proba(val)

# Матрица ошибок (или confusion matrix)
#                    	Предсказан класс 0	Предсказан класс 1
# Фактически класс 0	True Negative (TN)	False Positive (FP) 
# Фактически класс 1	False Negative (FN)	True Positive (TP)
# Ошибка первого рода (False Positive, FP) - err1 
# Ошибка второго рода (False Negative — FN) - err2
confusion_matrix_data = confusion_matrix(y_test, prediction, normalize='all')
_, err1, _, err2 = confusion_matrix_data.ravel()
print(prediction, probas)
print (err1, err2)
print(confusion_matrix_data)

[0 1 0 1 1 1 1 1 0 0 0 0 1 1 0 0 0 1 1 1] [[0.91319341 0.08680659]
 [0.25716922 0.74283078]
 [0.9465609  0.0534391 ]
 [0.02394008 0.97605992]
 [0.05253896 0.94746104]
 [0.3733327  0.6266673 ]
 [0.40603303 0.59396697]
 [0.15075113 0.84924887]
 [0.87996671 0.12003329]
 [0.64703817 0.35296183]
 [0.98295164 0.01704836]
 [0.64188079 0.35811921]
 [0.33222729 0.66777271]
 [0.27279345 0.72720655]
 [0.94561253 0.05438747]
 [0.95772252 0.04227748]
 [0.83146972 0.16853028]
 [0.47395417 0.52604583]
 [0.32721864 0.67278136]
 [0.00472072 0.99527928]]
0.2 0.35
[[0.45 0.2 ]
 [0.   0.35]]


In [5]:
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score,log_loss, confusion_matrix
# словарь со всеми метриками
metrics = {}

# посчитайте метрики из модуля sklearn.metrics
# err_1 — ошибка первого рода
# err_2 — ошибка второго рода

confusion_matrix_data = confusion_matrix(y_test, prediction, normalize='all')
_, err1, _, err2 = confusion_matrix_data.ravel()
auc = roc_auc_score(y_test, probas[:,1])
precision = precision_score(y_test, prediction)
recall = recall_score(y_test, prediction)
f1 = f1_score(y_test, prediction)
logloss = log_loss(y_test, prediction)

# запишите значения метрик в словарь
metrics["err1"] = err1
metrics["err2"] = err2
metrics["auc"] = auc
metrics["precision"] = precision
metrics["recall"] = recall
metrics["f1"] = f1
metrics["logloss"] = logloss
metrics

{'err1': 0.2,
 'err2': 0.35,
 'auc': 0.9340659340659341,
 'precision': 0.6363636363636364,
 'recall': 1.0,
 'f1': 0.7777777777777778,
 'logloss': 7.20873067782343}

In [7]:
model

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('binary',
                                                  OneHotEncoder(drop='if_binary'),
                                                  ['paperless_billing',
                                                   'internet_service',
                                                   'online_security',
                                                   'online_backup',
                                                   'device_protection',
                                                   'tech_support',
                                                   'streaming_tv',
                                                   'streaming_movies', 'gender',
                                                   'partner', 'dependents',
                                                   'multiple_lines']),
                                                 ('cat',
                                                  CatBoostEncoder(return_df=False),
                                                  ['begin_date', 'type',
                                                   'payment_method']),
                                                 ('num', StandardScaler(),
                                                  ['end_date',
                                                   'monthly_charges',
                                                   'total_charges'])],
                                   verbose_feature_names_out=False)),
                ('model',
                 <catboost.core.CatBoostClassifier object at 0x7fbb7c3ecc10>)])

##### Запись модели

In [ ]:
import os
import mlflow

from dotenv import load_dotenv
load_dotenv()

EXPERIMENT_NAME = 'bvv_43'# ваш код здесь (напишите своё уникальное имя эксперимента)
RUN_NAME = "model_0_registry"
REGISTRY_MODEL_NAME = "churn_model_bvv43"


# определяем основные credentials, которые нужны для подключения к MLflow
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "https://storage.yandexcloud.net" #endpoint бакета от YandexCloud
os.environ["AWS_ACCESS_KEY_ID"] = os.getenv("AWS_ACCESS_KEY_ID") # получаем id ключа бакета, к которому подключён MLFlow, из .env
os.environ["AWS_SECRET_ACCESS_KEY"] = os.getenv("AWS_SECRET_ACCESS_KEY") # получаем ключ бакета, к которому подключён MLFlow, из .env

# определяем глобальные переменные # поднимаем MLflow локально
TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000

# название эксперимента и запуска (run) внутри него
EXPERIMENT_NAME = "bvv_43"
RUN_NAME = "log_model"

# устанавливаем host, который будет отслеживать наши эксперименты
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_registry_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}") 


pip_requirements = "./requirements.txt"
#signature =  mlflow.models.infer_signature(X_test, prediction)
signature =  mlflow.models.infer_signature(data.head(5), model.predict(data.head(5)))
#input_example = X_test[:10]
input_example = data.head(5)
metadata = {"model_type": "monthly"}


# создаём эксперимент если его нет и записываем в него тестовую информацию
if not mlflow.get_experiment_by_name(EXPERIMENT_NAME):
    experiment_id = mlflow.create_experiment(EXPERIMENT_NAME) 
else:
    experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    run_id = run.info.run_id
    # ваш код здесь
    model_info = mlflow.sklearn.log_model( 
        artifact_path='models',
        sk_model =model,
        registered_model_name=REGISTRY_MODEL_NAME,
        pip_requirements=pip_requirements,
        signature=signature,
        input_example=input_example,
        metadata=metadata
        #code_paths=code_paths
		)



/home/mle-user/mle_projects/mle-mlflow/.venv/lib/python3.10/site-packages/mlflow/models/signature.py:212: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  inputs = _infer_schema(model_input) if model_input is not None else None
Registered model 'churn_model_bvv43' already exists. Creating a new version of this model...
2025/12/27 17:05:14 INFO mlflow.tracking._model_registry.c

##### Чтение модели

In [8]:
import os
import mlflow

from dotenv import load_dotenv
load_dotenv()

EXPERIMENT_NAME = 'bvv_43'# ваш код здесь (напишите своё уникальное имя эксперимента)
RUN_NAME = "model_0_registry"
REGISTRY_MODEL_NAME = "churn_model_bvv43"


# определяем основные credentials, которые нужны для подключения к MLflow
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "https://storage.yandexcloud.net" #endpoint бакета от YandexCloud
os.environ["AWS_ACCESS_KEY_ID"] = os.getenv("AWS_ACCESS_KEY_ID") # получаем id ключа бакета, к которому подключён MLFlow, из .env
os.environ["AWS_SECRET_ACCESS_KEY"] = os.getenv("AWS_SECRET_ACCESS_KEY") # получаем ключ бакета, к которому подключён MLFlow, из .env

# определяем глобальные переменные # поднимаем MLflow локально
TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000

# название эксперимента и запуска (run) внутри него
EXPERIMENT_NAME = "bvv_43"
RUN_NAME = "log_model"

# устанавливаем host, который будет отслеживать наши эксперименты
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_registry_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}") 

loaded_model = mlflow.sklearn.load_model(model_uri=model_info.model_uri) 

NameError: name 'model_info' is not defined

In [13]:
import mlflow

logged_model = 'runs:/4ae53a84931e43ec89d2a743799d531c/models'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame.
model_predictions = loaded_model.predict(data)

assert model_predictions.dtype == int

print(model_predictions[:10])

2025/12/27 11:45:36 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - scikit-learn (current: 1.4.0, required: scikit-learn==1.3.1)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


[0 1 0 1 1 1 1 1 0 0]


In [87]:
model

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('binary',
                                                  OneHotEncoder(drop='if_binary'),
                                                  ['paperless_billing',
                                                   'internet_service',
                                                   'online_security',
                                                   'online_backup',
                                                   'device_protection',
                                                   'tech_support',
                                                   'streaming_tv',
                                                   'streaming_movies', 'gender',
                                                   'partner', 'dependents',
                                                   'multiple_lines']),
                                                 ('cat',
                                                  CatBoostEncoder(return_df=False),
                                                  ['begin_date', 'type',
                                                   'payment_method']),
                                                 ('num', StandardScaler(),
                                                  ['end_date',
                                                   'monthly_charges',
                                                   'total_charges'])],
                                   verbose_feature_names_out=False)),
                ('model',
                 <catboost.core.CatBoostClassifier object at 0x7f5a31d03ee0>)])